In [8]:
import ecephys_analyses as ea
from ecephys.sglx_utils import load_trigger
from pyedflib import highlevel as edf
import numpy as np
from scipy.interpolate import interp1d
import xarray as xr

In [9]:
def resample(sig, target_fs):
    f = interp1d(sig.time, sig.T, kind='cubic')
    new_times = np.arange(sig.time.min(), sig.time.max(), 1/target_fs)
    new_data = f(new_times)
    return (new_times, new_data)

In [10]:
def write_edf(sig, emg, edf_path):
    target_fs = 100.0
    sig_time_rs, sig_data_rs = resample(sig, target_fs)
    emg_time_rs, emg_data_rs = resample(emg, target_fs)
    
    signals = np.vstack([sig_data_rs, emg_data_rs])
    signal_labels = [f"LF{ch}" for ch in sig.channel.values] + ["dEMG"]
    
    signal_headers = edf.make_signal_headers(signal_labels, dimension=sig.units, sample_rate=target_fs, physical_min=signals.min(), physical_max=signals.max())
    edf.write_edf(str(edf_path), signals, signal_headers)

In [11]:
def write_condition_edfs(subject, experiment, condition, probe):
    chans = ea.get_channels(subject, experiment, probe, 'scoring')
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="lf.bin")
    emg_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="emg.nc")
    edf_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="edf")
    
    for bin_path, emg_path, edf_path in zip(bin_paths, emg_paths, edf_paths):
        sig = load_trigger(bin_path, chans)
        emg = xr.load_dataarray(emg_path)
        write_edf(sig, emg, edf_path)

In [12]:
import matplotlib.pyplot as plt

def check_lfp_resampling(sig, time_rs, data_rs, n_seconds=4, iCh=0):
    fig, ax = plt.subplots(figsize=(36, 3))
    ax.plot(sig.time.values[:int(n_seconds*sig.fs)], sig.T.values[iCh, :int(n_seconds*sig.fs)])
    ax.plot(time_rs[:int(n_seconds*target_fs)], data_rs[iCh, :int(n_seconds*target_fs)])
    
def check_emg_resampling(emg, time_rs, data_rs, n_seconds=4, iCh=0):
    fig, ax = plt.subplots(figsize=(36, 3))
    ax.plot(emg.time.values[:int(n_seconds*emg.target_sf)], emg.values[:int(n_seconds*emg.target_sf)])
    ax.plot(time_rs[:int(n_seconds*target_fs)], data_rs[:int(n_seconds*target_fs)])

In [13]:
write_condition_edfs("Doppio", "ketamine-120mg", "all", "imec1")

nChan: 385, nFileSamp: 9000060


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pyedflib/highlevel.py:487: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically ' \


nChan: 385, nFileSamp: 9000059


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pyedflib/highlevel.py:487: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically ' \


nChan: 385, nFileSamp: 9000059


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pyedflib/highlevel.py:487: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically ' \


nChan: 385, nFileSamp: 9000060


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pyedflib/highlevel.py:487: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically ' \
